# Evolver Loop 3 Analysis

## Goal: Verify overlap issue and determine next steps

The evaluator claims the C++ optimizer has a buggy overlap detection that only checks edge-edge intersections, missing polygon containment. Let's verify this.

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
import warnings
warnings.filterwarnings('ignore')

# Tree geometry
TX = np.array([0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125])
TY = np.array([0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5])

def get_tree_polygon(x, y, deg):
    """Create a tree polygon at position (x, y) with rotation deg."""
    # Rotate
    rad = np.radians(deg)
    cos_a, sin_a = np.cos(rad), np.sin(rad)
    rx = TX * cos_a - TY * sin_a + x
    ry = TX * sin_a + TY * cos_a + y
    return Polygon(zip(rx, ry))

def load_submission(path):
    """Load submission and parse coordinates."""
    df = pd.read_csv(path)
    df['n'] = df['id'].str[:3].astype(int)
    df['idx'] = df['id'].str[4:].astype(int)
    # Remove 's' prefix if present
    for col in ['x', 'y', 'deg']:
        df[col] = df[col].astype(str).str.lstrip('s').astype(float)
    return df

print("Functions defined successfully")

In [ ]:
# Load both submissions
baseline = load_submission('/home/code/submission_candidates/candidate_000.csv')
optimized = load_submission('/home/code/experiments/004_cpp_optimizer/optimized20.csv')

print(f"Baseline shape: {baseline.shape}")
print(f"Optimized shape: {optimized.shape}")

In [ ]:
# Check for overlaps in the optimized submission for small N values
def check_overlaps(df, n_value):
    """Check if any trees overlap for a given N value."""
    subset = df[df['n'] == n_value].sort_values('idx')
    if len(subset) == 0:
        return None, 0
    
    polygons = []
    for _, row in subset.iterrows():
        poly = get_tree_polygon(row['x'], row['y'], row['deg'])
        polygons.append(poly)
    
    overlaps = []
    total_intersection = 0
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]):
                intersection = polygons[i].intersection(polygons[j])
                if intersection.area > 1e-10:  # Non-trivial overlap
                    overlaps.append((i, j, intersection.area))
                    total_intersection += intersection.area
    
    return overlaps, total_intersection

# Check N=2 to N=10 for overlaps
print("Checking optimized submission for overlaps:")
print("="*50)
for n in range(2, 11):
    overlaps, total_area = check_overlaps(optimized, n)
    if overlaps:
        print(f"N={n}: OVERLAP DETECTED! {len(overlaps)} pairs, total area={total_area:.6f}")
        for i, j, area in overlaps[:3]:  # Show first 3
            print(f"  Trees {i} and {j}: intersection area = {area:.6f}")
    else:
        print(f"N={n}: No overlaps")

print("\nChecking baseline submission for overlaps:")
print("="*50)
for n in range(2, 11):
    overlaps, total_area = check_overlaps(baseline, n)
    if overlaps:
        print(f"N={n}: OVERLAP DETECTED! {len(overlaps)} pairs, total area={total_area:.6f}")
    else:
        print(f"N={n}: No overlaps")

In [ ]:
# Calculate actual scores for both submissions
def calculate_score(df):
    """Calculate the total score for a submission."""
    scores = {}
    for n in range(1, 201):
        subset = df[df['n'] == n]
        if len(subset) == 0:
            continue
        
        # Get all tree polygons
        all_x, all_y = [], []
        for _, row in subset.iterrows():
            poly = get_tree_polygon(row['x'], row['y'], row['deg'])
            coords = np.array(poly.exterior.coords)
            all_x.extend(coords[:, 0])
            all_y.extend(coords[:, 1])
        
        # Bounding box
        min_x, max_x = min(all_x), max(all_x)
        min_y, max_y = min(all_y), max(all_y)
        side = max(max_x - min_x, max_y - min_y)
        
        scores[n] = side * side / n
    
    return scores

print("Calculating scores...")
baseline_scores = calculate_score(baseline)
optimized_scores = calculate_score(optimized)

print(f"\nBaseline total: {sum(baseline_scores.values()):.6f}")
print(f"Optimized total: {sum(optimized_scores.values()):.6f}")

# Compare scores for N=2-10
print("\nScore comparison for N=2-10:")
print(f"{'N':>3} | {'Baseline':>10} | {'Optimized':>10} | {'Diff':>10}")
print("-"*45)
for n in range(2, 11):
    b = baseline_scores.get(n, 0)
    o = optimized_scores.get(n, 0)
    diff = o - b
    print(f"{n:>3} | {b:>10.6f} | {o:>10.6f} | {diff:>+10.6f}")

In [ ]:
# Check if bbox3 binary is available and working
import subprocess

bbox3_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/bbox3'
result = subprocess.run(['ls', '-la', bbox3_path], capture_output=True, text=True)
print("bbox3 binary:")
print(result.stdout)
print(result.stderr)

# Try to run bbox3 with --help or version
result = subprocess.run([bbox3_path, '--help'], capture_output=True, text=True)
print("\nbbox3 help:")
print(result.stdout[:500] if result.stdout else "No output")
print(result.stderr[:500] if result.stderr else "No error")

In [ ]:
# Check GLIBC version
import subprocess
result = subprocess.run(['ldd', '--version'], capture_output=True, text=True)
print("System GLIBC version:")
print(result.stdout.split('\n')[0])

# Check what GLIBC bbox3 needs
result = subprocess.run(['ldd', bbox3_path], capture_output=True, text=True)
print("\nbbox3 dependencies:")
print(result.stdout[:1000] if result.stdout else "No output")
print(result.stderr[:500] if result.stderr else "")